<a href="https://colab.research.google.com/github/anandaiml19/Mlzoomcamp-Article/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Import Necessary Libraries**

In [ ]:
import pandas as pd

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

### **Dataset Import**

In [ ]:
ds=pd.read_parquet("/content/drive/MyDrive/0000.parquet")

In [ ]:
ds

,question,answer
0,Why whenever I get in the shower my girlfriend...,Isn’t it awful? You would swear that there was...
1,"What is a proxy, and how can I use one?",A proxy server is a system or router that prov...
2,"What song has the lyrics ""someone left the cak...",MacArthur's Park\n
3,I am the owner of an adult website called http...,Don't let apps that are liers put adds on your...
4,Does the Bible mention anything about a place ...,St. John in the book of Revelation mentions an...
...,...,...
56397,"Alexandria Ocasio-Cortez said ""Going by track ...","I think she’s right, one is a homosexual with ..."
56398,Is becoming a doctor financially worth it?,Yes if you want to help people and eliminate p...
56399,Where can one find the best biryani in bangalore?,Biryani crafts.These guys will give proper aut...
56400,Which smartphone is best for middle class people?,Oneplus nord\n[LINKED_TEXT: https://latesttech...


In [ ]:
print(type(ds))

<class 'pandas.core.frame.DataFrame'>


### **Dataset Preprocessing**

In [ ]:
ds.iloc[0,:]

question    Why whenever I get in the shower my girlfriend...
answer      Isn’t it awful? You would swear that there was...
Name: 0, dtype: object

In [ ]:
ds['ques_ans']=ds['question']+". " +ds['answer']

In [ ]:
ds

,question,answer,ques_ans
0,Why whenever I get in the shower my girlfriend...,Isn’t it awful? You would swear that there was...,Why whenever I get in the shower my girlfriend...
1,"What is a proxy, and how can I use one?",A proxy server is a system or router that prov...,"What is a proxy, and how can I use one?. A pro..."
2,"What song has the lyrics ""someone left the cak...",MacArthur's Park\n,"What song has the lyrics ""someone left the cak..."
3,I am the owner of an adult website called http...,Don't let apps that are liers put adds on your...,I am the owner of an adult website called http...
4,Does the Bible mention anything about a place ...,St. John in the book of Revelation mentions an...,Does the Bible mention anything about a place ...
...,...,...,...
56397,"Alexandria Ocasio-Cortez said ""Going by track ...","I think she’s right, one is a homosexual with ...","Alexandria Ocasio-Cortez said ""Going by track ..."
56398,Is becoming a doctor financially worth it?,Yes if you want to help people and eliminate p...,Is becoming a doctor financially worth it?. Ye...
56399,Where can one find the best biryani in bangalore?,Biryani crafts.These guys will give proper aut...,Where can one find the best biryani in bangalo...
56400,Which smartphone is best for middle class people?,Oneplus nord\n[LINKED_TEXT: https://latesttech...,Which smartphone is best for middle class peop...


### **Model Import**

In [ ]:
quora_embedding = SentenceTransformer('distiluse-base-multilingual-cased-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

### **Create Corpus Embeddings and Faiss Index**

In [ ]:
quoraqa_embeddings=quora_embedding.encode(ds['ques_ans'].values)
print(quoraqa_embeddings.shape)

(56402, 512)


In [ ]:
n_cells=1000
num_dimensions=quoraqa_embeddings.shape[1]
quantizer=faiss.IndexFlatL2(num_dimensions)
index=faiss.IndexIVFFlat(quantizer,num_dimensions,n_cells)
index.train(quoraqa_embeddings)
index.add(quoraqa_embeddings)

### **Query Embeddings and Semantic Search**

In [ ]:
query_text="aeroplanes"
query_embedding=quora_embedding.encode([query_text])
num_results=5
distances, indexes=index.search(query_embedding,num_results)

In [ ]:
indexes

array([[41434, 32781,   138,  7110, 50377]])

In [ ]:
rele_rows=ds.iloc[indexes[0]]
for i,row in rele_rows.iterrows():
  print("-",row['ques_ans'])

- In the aviation industry, what is your favorite pilot watch?. Types of Pilots servicing in the Aviation Industry.
 1).Helicopter pilot
 Those who can fly civilian and military helicopter.
 2). Fighter and Combat Pilot
 Pilots trained to fly supersonic fighter aircraft.
 3).Commercial pilots
 Highly experienced pilots flying passenger aircraft.
 4).Sport/Amphibian pilots
 Pilot capable of taking off and landing hydroplanes over water.
 5).Freighter pilots
 Crew who fly large Freighter aircraft.
 6).Balloonist
 Hobbyists who float in the atmosphere with hot air balloons.
 7).Surveillance pilots
 Extremely skilled military pilots who fly at extreme altitudes.
 8).Hobby aviators
 People who own and pilot their small, propeller aircraft.
 9).Test pilots
 Extremely skilled daredevils who test new models of aircraft.
 [LINKED_TEXT: Golden Epaulettes Aviation ] [URL: https://goldenepaulettes.com/how-to-become-a-pilot.php]is an aviation training institution that helps people in the aviation i

We searched using the query "Aeroplanes" and the query text is converted to query embeddings. The faiss index space is searched for text of high cosine similarity between the corpus and query embeddings and the ouput is returned.

As we can infer from the output that, it share close similarity with the word aeroplanes viz.,flight, military pilots, atmosphere, takeoff and landings.

### **Recomendation System**

In [ ]:
row_article=ds.iloc[28]

In [ ]:
row_article

question    Are the Iron Dome and David's Sling systems th...
answer      No, they aren’t. Hamas is a small organization...
ques_ans    Are the Iron Dome and David's Sling systems th...
Name: 28, dtype: object

In [ ]:
inp_artquery=row_article['question']

In [ ]:
query2_embedding=quora_embedding.encode([inp_artquery])
num_resultss=25
distancess, indexess=index.search(query2_embedding,num_resultss)

In [ ]:
rele_rowss=ds.iloc[indexess[0]]
for i,row in rele_rowss.iterrows():
  print("-",row['question'])

- Are the Iron Dome and David's Sling systems the most powerful air defense systems in the world, especially since Iran, Hamas Hezbollah have been unable for several years to manufacture planes and missiles that exceed these two systems?
- Are the Iron Dome and David's Sling systems the most powerful air defense systems in the world, especially since Iran, Hamas Hezbollah have been unable for several years to manufacture planes and missiles that exceed these two systems?
- Are the Iron Dome and David's Sling systems the most powerful air defense systems in the world, especially since Iran, Hamas Hezbollah have been unable for several years to manufacture planes and missiles that exceed these two systems?
- Are the Iron Dome and David's Sling systems the most powerful air defense systems in the world, especially since Iran, Hamas Hezbollah have been unable for several years to manufacture planes and missiles that exceed these two systems?
- Are the Iron Dome and David's Sling systems th

The above recommendation sysytem was able to suggest similar question on par with the input query. This dataset have repeated question,hence we see duplicates.